# DataCleaning Notebook - AAAG

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas_profiling
import numpy as np
import re
pd.set_option('display.max_columns', None)

In [4]:
df_train = pd.read_csv('train.csv',index_col ="Id")
df_train.shape

(1460, 80)

In [6]:
df_train.SalePrice = np.log1p(df_train.SalePrice)
df_train.sample(10)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
909,20,RL,NaN,8885,Pave,NaN,IR1,Low,AllPub,Inside,Mod,Mitchel,Norm,Norm,1Fam,1Story,5,5,1983,1983,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,CBlock,Gd,TA,Av,BLQ,301,ALQ,324,239,864,GasA,TA,Y,SBrkr,902,0,0,902,1,0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1983.0,Unf,2,484,TA,TA,Y,164,0,0,0,0,0,NaN,MnPrv,NaN,0,6,2006,WD,Normal,2.548113
544,120,RH,34.0,4058,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,TwnhsE,SFoyer,7,5,1998,1998,Gable,CompShg,MetalSd,MetalSd,BrkFace,182.0,TA,TA,PConc,Gd,TA,Av,GLQ,584,LwQ,139,0,723,GasA,Ex,Y,SBrkr,767,0,0,767,1,0,1,0,1,1,TA,4,Typ,0,NaN,Attchd,1998.0,Fin,1,367,TA,TA,Y,120,40,0,0,0,0,NaN,NaN,NaN,0,6,2007,WD,Normal,2.549298
691,120,RM,NaN,4426,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,TwnhsE,1Story,6,5,2004,2004,Gable,CompShg,VinylSd,VinylSd,BrkFace,147.0,Gd,TA,PConc,Gd,TA,Gd,GLQ,697,Unf,0,151,848,GasA,Ex,Y,SBrkr,848,0,0,848,1,0,1,0,1,1,Gd,3,Typ,1,TA,Attchd,2004.0,RFn,2,420,TA,TA,Y,149,0,0,0,0,0,NaN,NaN,NaN,0,5,2008,WD,Normal,2.553851
516,20,RL,94.0,12220,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,Norm,Norm,1Fam,1Story,10,5,2009,2009,Hip,CompShg,CemntBd,CmentBd,BrkFace,305.0,Ex,TA,CBlock,Ex,TA,No,GLQ,1436,Unf,0,570,2006,GasA,Ex,Y,SBrkr,2020,0,0,2020,1,0,2,1,3,1,Ex,9,Typ,1,Gd,Attchd,2009.0,Fin,3,900,TA,TA,Y,156,54,0,0,0,0,NaN,NaN,NaN,0,9,2009,New,Partial,2.632346
1394,190,RM,60.0,10800,Pave,Pave,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,2fmCon,1.5Fin,6,7,1905,2000,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,BrkTil,Fa,TA,No,Unf,0,Unf,0,482,482,GasA,Ex,N,SBrkr,1221,691,0,1912,0,0,2,0,3,2,TA,7,Typ,1,TA,Detchd,2003.0,Unf,2,672,Gd,TA,Y,0,25,212,0,0,0,NaN,NaN,NaN,0,4,2008,WD,Normal,2.565066
898,90,RL,64.0,7018,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,SawyerW,Feedr,Norm,Duplex,2Story,5,5,1979,1979,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,Slab,NaN,NaN,NaN,NaN,0,NaN,0,0,0,GasA,TA,Y,SBrkr,1120,1120,0,2240,0,0,2,0,6,2,TA,12,Typ,0,NaN,Detchd,1979.0,Unf,2,528,TA,TA,Y,154,0,0,0,0,0,NaN,NaN,NaN,0,6,2009,WD,Alloca,2.554921
902,20,RL,64.0,8712,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1957,2000,Hip,CompShg,MetalSd,MetalSd,None,0.0,TA,Gd,CBlock,TA,TA,Mn,BLQ,860,Unf,0,132,992,GasA,TA,Y,SBrkr,1306,0,0,1306,1,0,1,0,2,1,TA,5,Typ,0,NaN,Detchd,1968.0,Unf,1,756,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,5,2009,WD,Normal,2.560184
406,20,RL,NaN,9991,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Sawyer,Feedr,Norm,1Fam,1Story,4,4,1976,1993,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,CBlock,TA,TA,No,BLQ,1116,Unf,0,165,1281,GasA,Ex,Y,SBrkr,1620,0,0,1620,1,0,2,0,3,1,TA,8,Min1,1,TA,Attchd,1993.0,Unf,2,490,TA,TA,Y,120,78,0,0,0,0,NaN,GdWo,NaN,0,6,2009,WD,Normal,2.558652
411,20,RL,68.0,9571,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,1Story,5,3,1958,1958,Gable,CompShg,BrkComm,Brk Cmn,None,0.0,TA,Fa,CBlock,TA,Fa,No,Unf,0,Unf,0,1276,1276,GasA,TA,Y,FuseA,1276,0,0,1276,0,0,1,0,3,1,TA,5,Mod,0,NaN,Attchd,1958.0,Unf,1,350,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2009,COD,Abnorml,2.485083


In [4]:
#define function to dummify continuous porch, deck and pool variables
def make_binary(X,vrs):
    '''
    This is a mutating function that replaces any nonzero value with 1
    ----------
    X: dataframe
    vrs: list of variables or single variable
    '''
    for var in vrs:
        df_train.loc[df_train[var]>0,var]=1

In [5]:
def my_impute(df_col,method='replace',replace_with=None,missing_string=None,convert_to_numeric='infer'):
    '''
    - This function takes works for the columns mentioned above. 
    - df_col: take a column series 
    - replace: replace a null with a string defined by replace_with
    - replace_with: the string that replaces the null value
    - missing_string: define what constitutes a null value which will be replaced
    - convert_to_numeric: takes in "infer", "force", "no". "Infer" automatically senses what is 
    - the best action. Converts variable type to numeric. 
    '''
    new_series=df_col.copy()
    try:
        if (df_col.str.contains('Shed').any() and df_col.str.contains('TenC').any()):
            misc_dict={'Gar2':'Shed',
                      'Othr':'No Shed',
                      'TenC':'No Shed'}
            new_series.replace(misc_dict,inplace=True)
        if missing_string!=None:
            missing_dict={null_word:None for i in missing_string}
            new_series.replace(missing_dict,inplace=True) 
        if method=='replace':
            if replace_with==None:
                raise ValueError('Must provide replacement string!')
            new_series[new_series.isnull()]=replace_with
        elif method=='mode':
            new_series[new_series.isnull()]=new_series.mode(dropna=True)[0]
        if convert_to_numeric=='force': 
            score_dict={'TA':3,
                       'Gd':4,
                       'Ex':5,
                       'Fa':2,
                       'Po':1,
                       replace_with:0}
            new_series.replace(score_dict,inplace=True)
            return new_series
        elif convert_to_numeric=='infer':
            if new_series.str.contains('(^Gd$)',regex=True).any() and \
                new_series.str.contains('(^TA$)',regex=True).any() and \
                new_series.str.contains('(^Fa$)',regex=True).any():
                score_dict={'TA':3,
                           'Gd':4,
                           'Ex':5,
                           'Fa':2,
                           'Po':1,
                           replace_with:0}
                new_series.replace(score_dict,inplace=True)
                return new_series
            elif new_series.str.contains('(^Gd$)',regex=True).any() and \
                new_series.str.contains('(^Av$)',regex=True).any() and \
                new_series.str.contains('(^Mn$)',regex=True).any() and \
                new_series.str.contains('(^No$)',regex=True).any():
                score_dict={'Gd':4,
                           'Av':3,
                           'Mn':2,
                           'No':1,
                           replace_with:0}
                new_series.replace(score_dict,inplace=True)
                return new_series
            else:
                return new_series
        elif convert_to_numeric=='no':
            return new_series
        else:
            return new_series
    except:
        try: 
            if method=='mean':
                new_series[new_series.isnull()]=new_series.mean(skipna=True)
                return new_series
            elif method=='median':
                new_series[new_series.isnull()]=new_series.median(skipna=True)[0]
                return new_series
        except Exception as e:
            print(type(e),e)

In [6]:
# Clean:          
# - PavedDrive
# - SaleCondition
# - SaleType
# - Street
# - Central Air
# - Functional
# - Heating
# - 1stFlrSF
# - 2ndFlrSF
# - GrLivArea
# - LotArea
# - MSZone
# - OverallCond
# - OverallQual
# - TotalBsmtSF
# - YearBuilt
# - YearRemodAdd

# Drop
# - PoolQC
# - BsmtFinType2
# - Utilities
# - BsmtFinSF1  
# - BsmtFinSF2  
# - BsmtUnSF 
# - LowQualFinSF
# - KitchenAbvGr 
# - MiscVal 
df_train = df_train.drop(['PoolQC','BsmtFinType2','Utilities',\
                          'BsmtFinSF1', 'BsmtFinSF2',\
                          'BsmtUnfSF','LowQualFinSF','KitchenAbvGr', 'MiscVal'], axis=1)

# pply the make binary function to the porch, deck, and pool variables (MUTATING!)
vrs = ['3SsnPorch', 'EnclosedPorch', 'OpenPorchSF','PoolArea','ScreenPorch', 'WoodDeckSF']
make_binary(df_train,vrs)

# Converting to category/string
# - MSSubClass
# - MoSold
df_train['MSSubClass'] = df_train['MSSubClass'].astype('category')
df_train['MoSold'] = df_train['MoSold'].astype('category')

# Impute with mode/median
# - Electrical 
df_train.loc[:,'Electrical']=my_impute(df_train.loc[:,'Electrical'],method='mode',replace_with=None,missing_string=None,convert_to_numeric='no')

# Impute with mean 
# - GarageYrBlt
df_train.loc[:,'GarageYrBlt']=my_impute(df_train.loc[:,'GarageYrBlt'],method='mean',replace_with=None,missing_string=None,convert_to_numeric='no')

# Impute with groupedby median of neighborhood
# - LotFrontage
df_train['LotFrontage'] = df_train.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

# Replace with N/A with “no fence”, "No Alley" "No Basement" etc.
# - Fence
# - Alley
# - GarageFinish
# - GarageType
# - BsmtFinType1
df_train.loc[:,'Fence']=my_impute(df_train.loc[:,'Fence'],method='replace',replace_with='No Fence',missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'Alley']=my_impute(df_train.loc[:,'Alley'],method='replace',replace_with='No Alley',missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'GarageFinish']=my_impute(df_train.loc[:,'GarageFinish'],method='replace',replace_with='No Garage',missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'GarageType']=my_impute(df_train.loc[:,'GarageType'],method='replace',replace_with='No Garage',missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'BsmtFinType1']=my_impute(df_train.loc[:,'BsmtFinType1'],method='replace',replace_with='No Basement',missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'MasVnrType']=my_impute(df_train.loc[:,'MasVnrType'],method='replace',replace_with='No Masonry Veneer',missing_string=None,convert_to_numeric='no')

# Replace with N/A with 0 and turning ordinal into continuous variable
# - BsmtCond
# - BsmtExposure
# - BsmtQual
# - GarageCond
# - GarageQual
# - FireplaceQu
# - HeatingQC
# - KitchenQual
df_train.loc[:,'BsmtCond']=my_impute(df_train.loc[:,'BsmtCond'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'BsmtExposure']=my_impute(df_train.loc[:,'BsmtExposure'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'BsmtQual']=my_impute(df_train.loc[:,'BsmtQual'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'GarageCond']=my_impute(df_train.loc[:,'GarageCond'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'GarageQual']=my_impute(df_train.loc[:,'GarageQual'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'FireplaceQu']=my_impute(df_train.loc[:,'FireplaceQu'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'HeatingQC']=my_impute(df_train.loc[:,'HeatingQC'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'KitchenQual']=my_impute(df_train.loc[:,'KitchenQual'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'ExterCond']=my_impute(df_train.loc[:,'ExterCond'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')
df_train.loc[:,'ExterQual']=my_impute(df_train.loc[:,'ExterQual'],method='replace',replace_with=0,missing_string=None,convert_to_numeric='infer')

# Replace with N/A with 0 only
# - MasVnrArea
# - GarageArea
# - GarageCars
df_train['MasVnrArea'] = df_train['MasVnrArea'].fillna(0)
df_train['GarageArea'] = df_train['GarageArea'].fillna(0)
df_train['GarageCars'] = df_train['GarageCars'].fillna(0)

# Lumping features into shed and no shed
# - MiscFeature
df_train.loc[:,'MiscFeature']=my_impute(df_train.loc[:,'MiscFeature'],method='replace',replace_with='No Shed',missing_string=None,convert_to_numeric='infer')

# Check: 
df_train.isnull().sum().sum()

/home/auscheng/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


0

In [7]:
df_train.shape

(1460, 71)

In [8]:
df_train.to_csv('Version_5_df.csv')

In [7]:
df_train.Condition2.value_counts()

Norm      1445
Feedr        6
Artery       2
PosN         2
RRNn         2
PosA         1
RRAn         1
RRAe         1
Name: Condition2, dtype: int64